In [1]:
import numpy as np
from scipy.io import wavfile
from scipy import signal
import librosa
import librosa.core
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.image as img
import pylab
import os
from math import *

import IPython.display as ipd

from microfaune import audio
from microfaune import plot

# Format wav

In [ ]:
file_name =  'audio/test_cite_u.wav'
ipd.Audio(file_name)

In [ ]:
[spec, t, f, x_fs] = audio.file2spec(file_name, scale_spec = "linear", window_length=0.020, overlap=0.5, duration=10)
plot.plot_spec(spec, t, f, x_fs, scale_spec = "linear", plot_title="Spectrogram from wave file", fig_size=(20,5), save_fig=True, save_path="spec_wav.png")

# Format MP3

In [ ]:
file_name =  'audio/test_cite_u.mp3'
ipd.Audio(file_name)

In [ ]:
[spec, t, f, x_fs] = audio.file2spec(file_name, scale_spec = "linear", window_length=0.020, overlap=0.5, duration=10)
plot.plot_spec(spec, t, f, x_fs, scale_spec = "linear", plot_title="Spectrogram from mp3 file", fig_size=(20,5), save_fig=True, save_path="spec_mp3.png")

# Convert wav to mp3

In [ ]:
from pydub import AudioSegment

file = open('list_file_names.txt', 'r')
file.readline()
list_file_names = file.read()
list_file_names = list_file_names.split('\n')

for i in range(52): # select index in the species list
    file_name = list_file_names[i]
    
    old_path = 'audio/species_wav/' + file_name + '.wav'
    new_path = 'audio/species_mp3/' + file_name + '.mp3'
    
    
    AudioSegment.from_wav(old_path).export(new_path, format="mp3")
    